In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import fits_to_pandas, pandas_to_fits, decode_str_columns, cross_match_data_frames
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
from astropy.coordinates import SkyCoord
import astropy.coordinates 
import astropy.units as u
import astroquery

%matplotlib inline

matplotlib settings set


matplotlib settings set


load DESI counterparts

In [3]:
desi = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match.gz_pkl', compression='gzip')
desi.reset_index(drop=True, inplace=True)
desi[['nway_desi_id', 'desi_ra', 'desi_dec']].to_csv(data_path+'desi_LH_coords.csv', index=False)
erosita_columns = list(desi.columns.values)
desi.head(10)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_231,5.167984,9.665420e-01,0.976823,0.942008,0.930541,0.877164,14,0,False,True,9011,231,606779,154.762154,56.504538,3.362170,4.618722,5.989522,16.031805,20.984790,50.180990,-3.493270,435.342250,248.158630,104.400180,3.761635,0.939089,0.001432,0.000018,21.087423,20.774010,20.520338,19.482046,19.191862,18.247932,NaN,70.151200,72.758970,61.198788,31.093582,20.335644,1.898610,-0.014626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0
1,SRGe J101903.4+563325,154.764302,56.556994,154.897407,50.039645,12.232697,16.660557,2.758328e-14,6.850312e-15,34.209091,8.495835,1133.034668,0.0,0.0,0.0,4806,3902,154.766311,56.555705,5.407337,163.025894,0.000029,6.268413e+08,6.270701e+08,0.0,0.0,0.030192,0.007498,6.562579,154.764302,56.556994,-1,-1.000000,0,-1,NaN,-1.0,-1,SDSS J101904.67+563333.2,1.464000,QSO,13.943999,1,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,68265812198,5.407337,4.373272,6.601858,10.704663,132.318989,42.230639,2.122641e-14,1.507868e+20,9011_606779_284,12.929691,4.530361e-01,0.835886,0.953387,0.928812,0.577701,9,0,False,False,9011,284,606779,154.769507,56.559155,4.110681,6.741001,7.775545,18.952187,29.332830,65.296720,371.384800,259.819270,99.752320,72.978060,3.654050,0.866400,0.001278,0.000015,20.872555,20.365770,20.238258,19.300547,18.828358,17.962076,16.075178,66.259700,67.326480,66.424300,36.228188,27.303180,2.334288,1.453649,-0.009351,1.285011,-0.184517,0.341205,-2.763030,0.601020,G2,853371518808768256
2,SRGe J101911.0+562639,154.795822,56.444182,155.033709,50.112976,7.855049,18.953989,1.654359e-14,3.709296e-15,40.391277,9.056271,2230.516357,0.0,0.0,0.0,4530,3655,154.797830,56.442893,3.409649,142.069946,0.000015,6.268413e+08,6.270701e+08,0.0,0.0,0.018108,0.004060,12.856621,154.795822,56.444182,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,68713641563,3.409649,2.808233,4.239287,6.873844,132.413453,42.140474,1.273093e-14,1.537238e+20,9011_606779_455,15.145586,5.746393e-07,0.994162,-99.000000,-99.000000,0.230064,7,0,True,False,9011,455,606779,154.798428,56.448135,0.261824,0.279961

# Gaia cross match with epoch correction

In [4]:
gaia = pd.read_csv(data_path+'gaia_dr3_astroph_4-result.csv')

In [344]:

def gaia_coord_correction(df: pd.DataFrame,
                          s_n_threshold: int,
                          epochs_dict: dict = {'desi_epoch': 2000, 'ero_epoch': 2019.8},
                          replace_original_coords: bool = False) -> pd.DataFrame:
    """
    source: https://github.com/mbelveder/luminosity_LH/blob/ab9529c3ee2067a4060c1c3175f8a509ad1c6605/lh_functions.py#L325
    Corrects Gaia coordinates for the proper motion.
    ra  + 1. / 3600e3 * pmra_gaia  * (epoch - 2015.5) / cos(radians(dec)) as ra_gaia,
    dec + 1. / 3600e3 * pmdec_gaia * (epoch - 2015.5) as dec_gaia
    ref: https://www.cosmos.esa.int/web/gaia-users/archive/combine-with-other-data
    Args:
        df (pd.DataFrame): Gaia catalogue.
        epoch (float): Epoch for which coordinates will be propagated.
        s_n_threshold (int): S/N threshold for pmra and pmdec.
        replace_original_coords (bool): If True, replaces original Gaia coordinates with corrected ones.
    Returns:
        pd.DataFrame: Gaia catalogue with corrected coordinates.
    """
    gaia_epoch = 2016

    print(f'Gaia epoch: {gaia_epoch}')

    # Select only reliable pmra and pmdec (allows avoiding correction on noise)
    pmra_sn = df['pmra'].abs() / df['pmra_error']
    df['reliable_pmra'] = np.where(pmra_sn > s_n_threshold, df['pmra'], np.nan)

    pmdec_sn = df['pmdec'].abs() / df['pmdec_error']
    df['reliable_pmdec'] = np.where(pmdec_sn > s_n_threshold, df['pmdec'], np.nan)

    for postfix, epoch in epochs_dict.items():

        print()
        print(f'Target epoch: {epoch}')

        ra_prop_name = f'ra_{postfix}'
        dec_prop_name = f'dec_{postfix}'

        # Positional corrections for objrcts with reliable proper motion measurements
        df[ra_prop_name] = df['ra']  + df['reliable_pmra'] / 3600e3  * (epoch - gaia_epoch)\
                                    / np.cos(np.radians(df['dec']))
        df[dec_prop_name] = df['dec'] + df['reliable_pmdec'] / 3600e3 * (epoch - gaia_epoch)

        # Fill all NaN with original coordinates
        df[ra_prop_name] = df[ra_prop_name].fillna(df['ra'])
        df[dec_prop_name] = df[dec_prop_name].fillna(df['dec'])


        print(f'Column names: {ra_prop_name}, {dec_prop_name}, sep_{postfix}')

    if replace_original_coords:
        df['ra'] = df[ra_prop_name]
        df['dec'] = df[dec_prop_name]

    return df

gaia = gaia_coord_correction(gaia, s_n_threshold=3, epochs_dict={'desi_epoch': 2000})

Gaia epoch: 2016

Target epoch: 2000
Column names: ra_desi_epoch, dec_desi_epoch, sep_desi_epoch


In [345]:
desi_gaia = cross_match_data_frames(desi, gaia, 
                                colname_ra1='desi_ra',
                                colname_dec1='desi_dec',
                                colname_ra2='ra_desi_epoch',
                                colname_dec2='dec_desi_epoch',
                                match_radius = 1,
                                df_prefix = 'GAIA',
                                closest=True)
len(desi_gaia)

cross-match radius 1 arcsec
total matches: 2814 out of 9215
total closest matches: 2798


2798

# XMATCH

## with SDSS DR16

In [346]:
sdssq = pd.read_pickle(data_path+'SDSS_DR16Q_v4_LH.pkl')
sdssq.SDSS_NAME = sdssq.SDSS_NAME.str.decode('utf-8')
sdssq.SDSS_NAME =  'SDSS J' + sdssq.SDSS_NAME.astype(str)
desi_sdss = cross_match_data_frames(desi, sdssq, 
                                colname_ra1='desi_ra',
                                colname_dec1='desi_dec',
                                colname_ra2='RA',
                                colname_dec2='DEC',
                                match_radius = 1,
                                df_prefix = 'SDSS',
                                closest=True)
desi_sdss


cross-match radius 1 arcsec
total matches: 2017 out of 9215
total closest matches: 2017


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,SDSS_sep,SDSS_CATALOG,SDSS_SDSS_NAME,SDSS_RA,SDSS_DEC,SDSS_PLATE,SDSS_MJD,SDSS_FIBERID,SDSS_AUTOCLASS_PQN,SDSS_AUTOCLASS_DR14Q,SDSS_IS_QSO_QN,SDSS_Z_QN,SDSS_RANDOM_SELECT,SDSS_Z_10K,SDSS_Z_CONF_10K,SDSS_PIPE_CORR_10K,SDSS_IS_QSO_10K,SDSS_THING_ID,SDSS_Z_VI,SDSS_Z_CONF,SDSS_CLASS_PERSON,SDSS_Z_DR12Q,SDSS_IS_QSO_DR12Q,SDSS_Z_DR7Q_SCH,SDSS_IS_QSO_DR7Q,SDSS_Z_DR6Q_HW,SDSS_Z_DR7Q_HW,SDSS_IS_QSO_FINAL,SDSS_Z,SDSS_SOURCE_Z,SDSS_Z_PIPE,SDSS_ZWARNING,SDSS_OBJID,SDSS_Z_PCA,SDSS_ZWARN_PCA,SDSS_DELTACHI2_PCA,SDSS_Z_HALPHA,SDSS_ZWARN_HALPHA,SDSS_DELTACHI2_HALPHA,SDSS_Z_HBETA,SDSS_ZWARN_HBETA,SDSS_DELTACHI2_HBETA,SDSS_Z_MGII,SDSS_ZWARN_MGII,SDSS_DELTACHI2_MGII,SDSS_Z_CIII,SDSS_ZWARN_CIII,SDSS_DELTACHI2_CIII,SDSS_Z_CIV,SDSS_ZWARN_CIV,SDSS_DELTACHI2_CIV,SDSS_Z_LYA,SDSS_ZWARN_LYA,SDSS_DELTACHI2_LYA,SDSS_Z_LYAWG,SDSS_BAL_PROB,SDSS_BI_CIV,SDSS_ERR_BI_CIV,SDSS_AI_CIV,SDSS_ERR_AI_CIV,SDSS_BI_SIIV,SDSS_ERR_BI_SIIV,SDSS_AI_SIIV,SDSS_ERR_AI_SIIV,SDSS_BOSS_TARGET1,SDSS_EBOSS_TARGET0,SDSS_EBOSS_TARGET1,SDSS_EBOSS_TARGET2,SDSS_ANCILLARY_TARGET1,SDSS_ANCILLARY_TARGET2,SDSS_NSPEC_SDSS,SDSS_NSPEC_BOSS,SDSS_NSPEC,SDSS_SKYVERSION,SDSS_RUN_NUMBER,SDSS_RERUN_NUMBER,SDSS_CAMCOL_NUMBER,SDSS_FIELD_NUMBER,SDSS_ID_NUMBER,SDSS_LAMBDA_EFF,SDSS_ZOFFSET,SDSS_XFOCAL,SDSS_YFOCAL,SDSS_CHUNK,SDSS_TILE,SDSS_PLATESN2,SDSS_M_I,SDSS_SN_MEDIAN_ALL,SDSS_GALEX_MATCHED,SDSS_FUV,SDSS_FUV_IVAR,SDSS_NUV,SDSS_NUV_IVAR,SDSS_UKIDSS_MATCHED,SDSS_YFLUX,SDSS_YFLUX_ERR,SDSS_JFLUX,SDSS_JFLUX_ERR,SDSS_HFLUX,SDSS_HFLUX_ERR,SDSS_KFLUX,SDSS_KFLUX_ERR,SDSS_W1_FLUX,SDSS_W1_FLUX_IVAR,SDSS_W1_MAG,SDSS_W1_MAG_ERR,SDSS_W1_CHI2,SDSS_W1_FLUX_SNR,SDSS_W1_SRC_FRAC,SDSS_W1_EXT_FLUX,SDSS_W1_EXT_FRAC,SDSS_W1_NPIX,SDSS_W2_FLUX,SDSS_W2_FLUX_IVAR,SDSS_W2_MAG,SDSS_W2_MAG_ERR,SDSS_W2_CHI2,SDSS_W2_FLUX_SNR,SDSS_W2_SRC_FRAC,SDSS_W2_EXT_FLUX,SDSS_W2_EXT_FRAC,SDSS_W2_NPIX,SDSS_FIRST_MATCHED,SDSS_FIRST_FLUX,SDSS_FIRST_SNR,SDSS_SDSS2FIRST_SEP,SDSS_JMAG,SDSS_JMAG_ERR,SDSS_JSNR,SDSS_JRDFLAG,SDSS_HMAG,SDSS_HMAG_ERR,SDSS_HSNR,SDSS_HRDFLAG,SDSS_KMAG,SDSS_KMAG_ERR,SDSS_KSNR,SDSS_KRDFLAG,SDSS_SDSS2MASS_SEP,SDSS_2RXS_ID,SDSS_2RXS_RA,SDSS_2RXS_DEC,SDSS_2RXS_SRC_FLUX,SDSS_2RXS_SRC_FLUX_ERR,SDSS_SDSS2ROSAT_SEP,SDSS_XMM_SRC_ID,SDSS_XMM_RA,SDSS_XMM_DEC,SDSS_XMM_SOFT_FLUX,SDSS_XMM_SOFT_FLUX_ERR,SDSS_XMM_HARD_FLUX,SDSS_XMM_HARD_FLUX_ERR,SDSS_XMM_TOTAL_FLUX,SDSS_XMM_TOTAL_FLUX_ERR,SDSS_XMM_TOTAL_LUM,SDSS_SDSS2XMM_SEP,SDSS_GAIA_MATCHED,SDSS_GAIA_DESIGNATION,SDSS_GAIA_RA,SDSS_GAIA_DEC,SDSS_GAIA_PARALLAX,SDSS_GAIA_PARALLAX_ERR,SDSS_GAIA_PM_RA,SDSS_GAIA_PM_RA_ERR,SDSS_GAIA_PM_DEC,SDSS_GAIA_PM_DEC_ERR,SDSS_GAIA_G_MAG,SDSS_GAIA_G_FLUX_SNR,SDSS_GAIA_BP_MAG,SDSS_GAIA_BP_FLUX_SNR,SDSS_GAIA_RP_MAG,SDSS_GAIA_RP_FLUX_SNR,SDSS_SDSS2GAIA_SEP
0,SRGe J101903.3+

https://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=V/154

In [347]:
#TODO ADD SDSS DR16 FROM XMATCH
# xmatch_sdss = pd.read_csv(data_path+'desi_LH_coords_sdssdr16.csv')
# print(len(xmatch_sdss), 'sdss sources after cross-match')
# xmatch_sdss = xmatch_sdss.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
# xmatch_sdss = xmatch_sdss.query('~spCl.isna()') #remove objects without classification
# print(len(xmatch_sdss), 'sdss sources the classification')
# xmatch_sdss.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
# new_cols = ['SDSS_'+x  if 'desi' not in x  else x  for x in xmatch_sdss.columns]
# xmatch_sdss.columns = new_cols
# print(len(xmatch_sdss), 'sdss sources the closest filter')
# xmatch_sdss
# xmatch_sdss.SDSS_spCl.value_counts()
# desi_sdss = desi.merge(xmatch_sdss, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')

## with SIMBAD

In [348]:
# xmatch_simbad = pd.read_csv(data_path+'desi_LH_coords_simbad.csv')
# xmatch_simbad = xmatch_simbad.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
# #xmatch_simbad.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
# new_cols = ['XMATCH_Simbad_'+x  if 'desi' not in x  else x  for x in xmatch_simbad.columns]
# xmatch_simbad.columns = new_cols
# xmatch_simbad

## with millquas

https://heasarc.gsfc.nasa.gov/W3Browse/all/milliquas.html


https://quasars.org/Milliquas-ReadMe.txt

In [349]:
milq = pd.read_csv(data_path+'milliquas_LH.csv', sep=',')
milq.NAME = milq.NAME.str.strip()
milq


,RA,DEC,NAME,TYPE,RMAG,BMAG,COMMENT,R,B,Z,CITE,ZCITE,RXPCT,QPCT,XNAME,RNAME,LOBE1,LOBE2
0,154.764290,58.221086,SDSS J101903.42+581315.9,GR2,21.96,23.74,g,1,1,0.831,DR16,DR16,97.0,3,,FIRST J101903.4+581314,FIRST J101902.8+581302,FIRST J101904.8+581337
1,154.764559,56.914829,SDSS J101903.49+565453.3,Q,20.88,21.02,g,-,-,1.939,DR16Q,DR16Q,NaN,-32768,,,,
2,154.768121,55.474116,SDSS J101904.34+552826.8,Q,21.60,21.70,g,-,-,0.980,DR16Q,DR16Q,NaN,-32768,,,,
3,154.769509,56.559167,SDSS J101904.68+563332.9,Q,20.25,20.67,gG,-,-,1.464,DR16Q,DR16Q,NaN,-32768,,,,
4,154.769562,58.311203,SDSS J101904.69+581840.3,Q,21.10,21.41,g,-,-,1.205,DR16Q,DR16Q,NaN,-32768,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7722,167.079417,54.698561,SDSS J110819.06+544154.8,Q,21.59,22.18,g,-,-,1.571,DR16Q,DR16Q,NaN,-32768,,,,
7723,167.082690,56.258243,PGC 2530909,GR,15.75,18.20,p+,1,1,0.138,PGC,DR16,97.0,1,,FIRST J110819.9+561528,,
7724,167.082922,55.969132,SDSS J110819.90+555808.8,Q,21.06,21.31,g,-,-,0.929,DR16Q,DR16,NaN,-32768,,,,
7725,167.083031,55.387843,SDSS J110819.92+552316.3,Q,20.38,20.43,gG,1,1,1.832,DR16Q,DR16,NaN,-32768,,,,


In [350]:
desi_milq = cross_match_data_frames(desi, milq, 
                                colname_ra1='desi_ra',
                                colname_dec1='desi_dec',
                                colname_ra2='RA',
                                colname_dec2='DEC',
                                match_radius = 2,
                                df_prefix = 'MILQ',
                                closest=True)

desi_milq.MILQ_TYPE = desi_milq.MILQ_TYPE.str.strip()
desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.str.replace('R', '')
desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.str.replace('X', '')
desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.str.replace('2', '')
type_dict_milq = {
    'Q': 'QSO',
    'A': 'AGN',
    'B': 'BLAZAR',
    'L': 'QSO-LENSED',
    'K': 'QSO-NL',
    'N': 'AGN-NL',
    'G': 'GALAXY',
    'S': 'STAR',
    'C': 'CV',
    'M': 'MOVING-OBJECT',
    '': 'RADIO-SOURCE',
}


desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.map(type_dict_milq)


cross-match radius 2 arcsec
total matches: 2725 out of 9215
total closest matches: 2725


In [351]:
# #ADD XMATCH with MILQ
# # f = astropy.io.fits.open(data_path+'milliquas.fits')
# # mask_ra = (f[1].data['RA'] > desi[['RA_fin', 'DEC_fin']].min().RA_fin) & (f[1].data['RA'] < desi[['RA_fin', 'DEC_fin']].max().RA_fin)
# # mask_dec = (f[1].data['DEC'] > desi[['RA_fin', 'DEC_fin']].min().DEC_fin) & (f[1].data['DEC'] < desi[['RA_fin', 'DEC_fin']].max().DEC_fin)
# # mask = mask_ra & mask_dec
# # data_tmp = f[1].data[mask]
# # milq = pd.DataFrame(data_tmp)
# # milq.to_csv(data_path+'milliquas_LH.csv', index=False)
# #milq = pd.read_csv(data_path+'milliquas_LH.csv')
# xmatch_milq = pd.read_csv(data_path+'desi_LH_coords_milliquas.csv')
# xmatch_milq = xmatch_milq.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
# print('milq LH match (2arcsec)', len(xmatch_milq))

# xmatch_milq.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
# new_cols = ['MILQ_'+x  if 'desi' not in x  else x  for x in xmatch_milq.columns]
# xmatch_milq.columns = new_cols
# print('milq LH match (2arcsec) - closest', len(xmatch_milq))

# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('R', '')
# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('X', '')
# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('2', '')
# type_dict_milq = {
#     'Q': 'QSO',
#     'A': 'AGN',
#     'B': 'BLAZAR',
#     'L': 'QSO-LENSED',
#     'K': 'QSO-NL',
#     'N': 'AGN-NL',
#     'G': 'GALAXY',
#     'S': 'STAR',
#     'C': 'CV',
#     'M': 'MOVING-OBJECT',
#     '': 'RADIO-SOURCE',
# }
# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.map(type_dict_milq)
# desi_milq = desi.merge(xmatch_milq, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')

# xmatch_milq



### compare SDSS and MILQ

In [352]:
desi_sdss_milq = desi_sdss.merge(desi_milq, on=erosita_columns, how='right')
#test_cols = ['SDSS_angDist', 'MILQ_angDist','SDSS_SDSS16', 'SDSS_spCl', 'SDSS_subCl',  'MILQ_NAME', 'MILQ_TYPE', 'SDSS_zsp', 'MILQ_Z']

In [353]:
tmp_df = desi_sdss_milq.query("~SDSS_SDSS_NAME.isna() & ~MILQ_NAME.isna() & (MILQ_NAME == SDSS_SDSS_NAME)  ")#[test_cols]
print('sdss -  milq complete match', len(tmp_df))
tmp_df.sample(5)

sdss -  milq complete match 1881


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,SDSS_sep,SDSS_CATALOG,SDSS_SDSS_NAME,SDSS_RA,SDSS_DEC,SDSS_PLATE,SDSS_MJD,SDSS_FIBERID,SDSS_AUTOCLASS_PQN,SDSS_AUTOCLASS_DR14Q,SDSS_IS_QSO_QN,SDSS_Z_QN,SDSS_RANDOM_SELECT,SDSS_Z_10K,SDSS_Z_CONF_10K,SDSS_PIPE_CORR_10K,SDSS_IS_QSO_10K,SDSS_THING_ID,SDSS_Z_VI,SDSS_Z_CONF,SDSS_CLASS_PERSON,SDSS_Z_DR12Q,SDSS_IS_QSO_DR12Q,SDSS_Z_DR7Q_SCH,SDSS_IS_QSO_DR7Q,SDSS_Z_DR6Q_HW,SDSS_Z_DR7Q_HW,SDSS_IS_QSO_FINAL,SDSS_Z,SDSS_SOURCE_Z,SDSS_Z_PIPE,SDSS_ZWARNING,SDSS_OBJID,SDSS_Z_PCA,SDSS_ZWARN_PCA,SDSS_DELTACHI2_PCA,SDSS_Z_HALPHA,SDSS_ZWARN_HALPHA,SDSS_DELTACHI2_HALPHA,SDSS_Z_HBETA,SDSS_ZWARN_HBETA,SDSS_DELTACHI2_HBETA,SDSS_Z_MGII,SDSS_ZWARN_MGII,SDSS_DELTACHI2_MGII,SDSS_Z_CIII,SDSS_ZWARN_CIII,SDSS_DELTACHI2_CIII,SDSS_Z_CIV,SDSS_ZWARN_CIV,SDSS_DELTACHI2_CIV,SDSS_Z_LYA,SDSS_ZWARN_LYA,SDSS_DELTACHI2_LYA,SDSS_Z_LYAWG,SDSS_BAL_PROB,SDSS_BI_CIV,SDSS_ERR_BI_CIV,SDSS_AI_CIV,SDSS_ERR_AI_CIV,SDSS_BI_SIIV,SDSS_ERR_BI_SIIV,SDSS_AI_SIIV,SDSS_ERR_AI_SIIV,SDSS_BOSS_TARGET1,SDSS_EBOSS_TARGET0,SDSS_EBOSS_TARGET1,SDSS_EBOSS_TARGET2,SDSS_ANCILLARY_TARGET1,SDSS_ANCILLARY_TARGET2,SDSS_NSPEC_SDSS,SDSS_NSPEC_BOSS,SDSS_NSPEC,SDSS_SKYVERSION,SDSS_RUN_NUMBER,SDSS_RERUN_NUMBER,SDSS_CAMCOL_NUMBER,SDSS_FIELD_NUMBER,SDSS_ID_NUMBER,SDSS_LAMBDA_EFF,SDSS_ZOFFSET,SDSS_XFOCAL,SDSS_YFOCAL,SDSS_CHUNK,SDSS_TILE,SDSS_PLATESN2,SDSS_M_I,SDSS_SN_MEDIAN_ALL,SDSS_GALEX_MATCHED,SDSS_FUV,SDSS_FUV_IVAR,SDSS_NUV,SDSS_NUV_IVAR,SDSS_UKIDSS_MATCHED,SDSS_YFLUX,SDSS_YFLUX_ERR,SDSS_JFLUX,SDSS_JFLUX_ERR,SDSS_HFLUX,SDSS_HFLUX_ERR,SDSS_KFLUX,SDSS_KFLUX_ERR,SDSS_W1_FLUX,SDSS_W1_FLUX_IVAR,SDSS_W1_MAG,SDSS_W1_MAG_ERR,SDSS_W1_CHI2,SDSS_W1_FLUX_SNR,SDSS_W1_SRC_FRAC,SDSS_W1_EXT_FLUX,SDSS_W1_EXT_FRAC,SDSS_W1_NPIX,SDSS_W2_FLUX,SDSS_W2_FLUX_IVAR,SDSS_W2_MAG,SDSS_W2_MAG_ERR,SDSS_W2_CHI2,SDSS_W2_FLUX_SNR,SDSS_W2_SRC_FRAC,SDSS_W2_EXT_FLUX,SDSS_W2_EXT_FRAC,SDSS_W2_NPIX,SDSS_FIRST_MATCHED,SDSS_FIRST_FLUX,SDSS_FIRST_SNR,SDSS_SDSS2FIRST_SEP,SDSS_JMAG,SDSS_JMAG_ERR,SDSS_JSNR,SDSS_JRDFLAG,SDSS_HMAG,SDSS_HMAG_ERR,SDSS_HSNR,SDSS_HRDFLAG,SDSS_KMAG,SDSS_KMAG_ERR,SDSS_KSNR,SDSS_KRDFLAG,SDSS_SDSS2MASS_SEP,SDSS_2RXS_ID,SDSS_2RXS_RA,SDSS_2RXS_DEC,SDSS_2RXS_SRC_FLUX,SDSS_2RXS_SRC_FLUX_ERR,SDSS_SDSS2ROSAT_SEP,SDSS_XMM_SRC_ID,SDSS_XMM_RA,SDSS_XMM_DEC,SDSS_XMM_SOFT_FLUX,SDSS_XMM_SOFT_FLUX_ERR,SDSS_XMM_HARD_FLUX,SDSS_XMM_HARD_FLUX_ERR,SDSS_XMM_TOTAL_FLUX,SDSS_XMM_TOTAL_FLUX_ERR,SDSS_XMM_TOTAL_LUM,SDSS_SDSS2XMM_SEP,SDSS_GAIA_MATCHED,SDSS_GAIA_DESIGNATION,SDSS_GAIA_RA,SDSS_GAIA_DEC,SDSS_GAIA_PARALLAX,SDSS_GAIA_PARALLAX_ERR,SDSS_GAIA_PM_RA,SDSS_GAIA_PM_RA_ERR,SDSS_GAIA_PM_DEC,SDSS_GAIA_PM_DEC_ERR,SDSS_GAIA_G_MAG,SDSS_GAIA_G_FLUX_SNR,SDSS_GAIA_BP_MAG,SDSS_GAIA_BP_FLUX_SNR,SDSS_GAIA_RP_MAG,SDSS_GAIA_RP_FLUX_SNR,SDSS_SDSS2GAIA_SEP,MILQ_sep,MILQ_RA,

In [354]:
tmp_df = desi_sdss_milq.query("~SDSS_SDSS_NAME.isna() & ~MILQ_NAME.isna() & (MILQ_NAME != SDSS_SDSS_NAME)  ")
tmp_df['z_diff'] = np.abs(tmp_df['MILQ_Z'] - tmp_df['SDSS_Z'])
tmp_df = tmp_df.sort_values(by=['z_diff'], ascending=False)
#max_z_diff = (tmp_df.MILQ_Z - tmp_df.SDSS_Z).max()
#print('max z diff', max_z_diff)
print('sdss -  milq un match -', len(tmp_df))
print('sdss -  milq un match - max z diff', tmp_df.z_diff.max())
print('sdss -  milq un match - z diff < 0.02', np.sum(tmp_df.z_diff<0.02))

#print(tmp_df.z_diff.describe())
tmp_df.head(15)

sdss -  milq un match - 118
sdss -  milq un match - max z diff 0.006000000000000227
sdss -  milq un match - z diff < 0.02 118


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,SDSS_sep,SDSS_CATALOG,SDSS_SDSS_NAME,SDSS_RA,SDSS_DEC,SDSS_PLATE,SDSS_MJD,SDSS_FIBERID,SDSS_AUTOCLASS_PQN,SDSS_AUTOCLASS_DR14Q,SDSS_IS_QSO_QN,SDSS_Z_QN,SDSS_RANDOM_SELECT,SDSS_Z_10K,SDSS_Z_CONF_10K,SDSS_PIPE_CORR_10K,SDSS_IS_QSO_10K,SDSS_THING_ID,SDSS_Z_VI,SDSS_Z_CONF,SDSS_CLASS_PERSON,SDSS_Z_DR12Q,SDSS_IS_QSO_DR12Q,SDSS_Z_DR7Q_SCH,SDSS_IS_QSO_DR7Q,SDSS_Z_DR6Q_HW,SDSS_Z_DR7Q_HW,SDSS_IS_QSO_FINAL,SDSS_Z,SDSS_SOURCE_Z,SDSS_Z_PIPE,SDSS_ZWARNING,SDSS_OBJID,SDSS_Z_PCA,SDSS_ZWARN_PCA,SDSS_DELTACHI2_PCA,SDSS_Z_HALPHA,SDSS_ZWARN_HALPHA,SDSS_DELTACHI2_HALPHA,SDSS_Z_HBETA,SDSS_ZWARN_HBETA,SDSS_DELTACHI2_HBETA,SDSS_Z_MGII,SDSS_ZWARN_MGII,SDSS_DELTACHI2_MGII,SDSS_Z_CIII,SDSS_ZWARN_CIII,SDSS_DELTACHI2_CIII,SDSS_Z_CIV,SDSS_ZWARN_CIV,SDSS_DELTACHI2_CIV,SDSS_Z_LYA,SDSS_ZWARN_LYA,SDSS_DELTACHI2_LYA,SDSS_Z_LYAWG,SDSS_BAL_PROB,SDSS_BI_CIV,SDSS_ERR_BI_CIV,SDSS_AI_CIV,SDSS_ERR_AI_CIV,SDSS_BI_SIIV,SDSS_ERR_BI_SIIV,SDSS_AI_SIIV,SDSS_ERR_AI_SIIV,SDSS_BOSS_TARGET1,SDSS_EBOSS_TARGET0,SDSS_EBOSS_TARGET1,SDSS_EBOSS_TARGET2,SDSS_ANCILLARY_TARGET1,SDSS_ANCILLARY_TARGET2,SDSS_NSPEC_SDSS,SDSS_NSPEC_BOSS,SDSS_NSPEC,SDSS_SKYVERSION,SDSS_RUN_NUMBER,SDSS_RERUN_NUMBER,SDSS_CAMCOL_NUMBER,SDSS_FIELD_NUMBER,SDSS_ID_NUMBER,SDSS_LAMBDA_EFF,SDSS_ZOFFSET,SDSS_XFOCAL,SDSS_YFOCAL,SDSS_CHUNK,SDSS_TILE,SDSS_PLATESN2,SDSS_M_I,SDSS_SN_MEDIAN_ALL,SDSS_GALEX_MATCHED,SDSS_FUV,SDSS_FUV_IVAR,SDSS_NUV,SDSS_NUV_IVAR,SDSS_UKIDSS_MATCHED,SDSS_YFLUX,SDSS_YFLUX_ERR,SDSS_JFLUX,SDSS_JFLUX_ERR,SDSS_HFLUX,SDSS_HFLUX_ERR,SDSS_KFLUX,SDSS_KFLUX_ERR,SDSS_W1_FLUX,SDSS_W1_FLUX_IVAR,SDSS_W1_MAG,SDSS_W1_MAG_ERR,SDSS_W1_CHI2,SDSS_W1_FLUX_SNR,SDSS_W1_SRC_FRAC,SDSS_W1_EXT_FLUX,SDSS_W1_EXT_FRAC,SDSS_W1_NPIX,SDSS_W2_FLUX,SDSS_W2_FLUX_IVAR,SDSS_W2_MAG,SDSS_W2_MAG_ERR,SDSS_W2_CHI2,SDSS_W2_FLUX_SNR,SDSS_W2_SRC_FRAC,SDSS_W2_EXT_FLUX,SDSS_W2_EXT_FRAC,SDSS_W2_NPIX,SDSS_FIRST_MATCHED,SDSS_FIRST_FLUX,SDSS_FIRST_SNR,SDSS_SDSS2FIRST_SEP,SDSS_JMAG,SDSS_JMAG_ERR,SDSS_JSNR,SDSS_JRDFLAG,SDSS_HMAG,SDSS_HMAG_ERR,SDSS_HSNR,SDSS_HRDFLAG,SDSS_KMAG,SDSS_KMAG_ERR,SDSS_KSNR,SDSS_KRDFLAG,SDSS_SDSS2MASS_SEP,SDSS_2RXS_ID,SDSS_2RXS_RA,SDSS_2RXS_DEC,SDSS_2RXS_SRC_FLUX,SDSS_2RXS_SRC_FLUX_ERR,SDSS_SDSS2ROSAT_SEP,SDSS_XMM_SRC_ID,SDSS_XMM_RA,SDSS_XMM_DEC,SDSS_XMM_SOFT_FLUX,SDSS_XMM_SOFT_FLUX_ERR,SDSS_XMM_HARD_FLUX,SDSS_XMM_HARD_FLUX_ERR,SDSS_XMM_TOTAL_FLUX,SDSS_XMM_TOTAL_FLUX_ERR,SDSS_XMM_TOTAL_LUM,SDSS_SDSS2XMM_SEP,SDSS_GAIA_MATCHED,SDSS_GAIA_DESIGNATION,SDSS_GAIA_RA,SDSS_GAIA_DEC,SDSS_GAIA_PARALLAX,SDSS_GAIA_PARALLAX_ERR,SDSS_GAIA_PM_RA,SDSS_GAIA_PM_RA_ERR,SDSS_GAIA_PM_DEC,SDSS_GAIA_PM_DEC_ERR,SDSS_GAIA_G_MAG,SDSS_GAIA_G_FLUX_SNR,SDSS_GAIA_BP_MAG,SDSS_GAIA_BP_FLUX_SNR,SDSS_GAIA_RP_MAG,SDSS_GAIA_RP_FLUX_SNR,SDSS_SDSS2GAIA_SEP,MILQ_sep,MILQ_RA,

In [355]:
# desi_sdss_milq.query("~SDSS_SDSS16.isna() & SDSS_spCl=='GALAXY'")[test_cols]

In [356]:
# xmatch_sdss_milq.query("~SDSS_SDSS16.isna() & MILQ_NAME.isna() & (MILQ_NAME != SDSS_SDSS16)  ")

In [357]:
# xmatch_sdss_milq.groupby('MILQ_TYPE')['SDSS_spCl', 'SDSS_subCl'].value_counts()

In [358]:
# xmatch_sdss_milq.groupby(['SDSS_spCl', 'SDSS_subCl'])['MILQ_TYPE'].value_counts()

# Join all matches

In [359]:
#merge desi_gaia with desi_sdss with desi to create a new desi_gaia_sdss catalog 
#https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
from functools import reduce
data_frames = [desi, desi_gaia, desi_sdss, desi_milq]

desi_matched = reduce(lambda  left,right: pd.merge(left,right,on=erosita_columns,
                                            how='outer'), data_frames).fillna(np.nan)

desi_matched.fillna(np.nan, inplace=True)

desi_matched

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,GAIA_sep,GAIA_source_id,GAIA_ra,GAIA_ra_error,GAIA_dec,GAIA_dec_error,GAIA_parallax,GAIA_parallax_error,GAIA_pmra,GAIA_pmra_error,GAIA_pmdec,GAIA_pmdec_error,GAIA_phot_g_mean_flux,GAIA_phot_g_mean_flux_error,GAIA_phot_g_mean_mag,GAIA_phot_bp_mean_flux,GAIA_phot_bp_mean_flux_error,GAIA_phot_bp_mean_mag,GAIA_phot_rp_mean_flux,GAIA_phot_rp_mean_flux_error,GAIA_phot_rp_mean_mag,GAIA_bp_rp,GAIA_classprob_dsc_combmod_quasar,GAIA_classprob_dsc_combmod_galaxy,GAIA_classprob_dsc_combmod_star,GAIA_classprob_dsc_combmod_whitedwarf,GAIA_classprob_dsc_combmod_binarystar,GAIA_spectraltype_esphs,GAIA_distance_gspphot,GAIA_distance_msc,GAIA_reliable_pmra,GAIA_reliable_pmdec,GAIA_ra_desi_epoch,GAIA_dec_desi_epoch,SDSS_sep,SDSS_CATALOG,SDSS_SDSS_NAME,SDSS_RA,SDSS_DEC,SDSS_PLATE,SDSS_MJD,SDSS_FIBERID,SDSS_AUTOCLASS_PQN,SDSS_AUTOCLASS_DR14Q,SDSS_IS_QSO_QN,SDSS_Z_QN,SDSS_RANDOM_SELECT,SDSS_Z_10K,SDSS_Z_CONF_10K,SDSS_PIPE_CORR_10K,SDSS_IS_QSO_10K,SDSS_THING_ID,SDSS_Z_VI,SDSS_Z_CONF,SDSS_CLASS_PERSON,SDSS_Z_DR12Q,SDSS_IS_QSO_DR12Q,SDSS_Z_DR7Q_SCH,SDSS_IS_QSO_DR7Q,SDSS_Z_DR6Q_HW,SDSS_Z_DR7Q_HW,SDSS_IS_QSO_FINAL,SDSS_Z,SDSS_SOURCE_Z,SDSS_Z_PIPE,SDSS_ZWARNING,SDSS_OBJID,SDSS_Z_PCA,SDSS_ZWARN_PCA,SDSS_DELTACHI2_PCA,SDSS_Z_HALPHA,SDSS_ZWARN_HALPHA,SDSS_DELTACHI2_HALPHA,SDSS_Z_HBETA,SDSS_ZWARN_HBETA,SDSS_DELTACHI2_HBETA,SDSS_Z_MGII,SDSS_ZWARN_MGII,SDSS_DELTACHI2_MGII,SDSS_Z_CIII,SDSS_ZWARN_CIII,SDSS_DELTACHI2_CIII,SDSS_Z_CIV,SDSS_ZWARN_CIV,SDSS_DELTACHI2_CIV,SDSS_Z_LYA,SDSS_ZWARN_LYA,SDSS_DELTACHI2_LYA,SDSS_Z_LYAWG,SDSS_BAL_PROB,SDSS_BI_CIV,SDSS_ERR_BI_CIV,SDSS_AI_CIV,SDSS_ERR_AI_CIV,SDSS_BI_SIIV,SDSS_ERR_BI_SIIV,SDSS_AI_SIIV,SDSS_ERR_AI_SIIV,SDSS_BOSS_TARGET1,SDSS_EBOSS_TARGET0,SDSS_EBOSS_TARGET1,SDSS_EBOSS_TARGET2,SDSS_ANCILLARY_TARGET1,SDSS_ANCILLARY_TARGET2,SDSS_NSPEC_SDSS,SDSS_NSPEC_BOSS,SDSS_NSPEC,SDSS_SKYVERSION,SDSS_RUN_NUMBER,SDSS_RERUN_NUMBER,SDSS_CAMCOL_NUMBER,SDSS_FIELD_NUMBER,SDSS_ID_NUMBER,SDSS_LAMBDA_EFF,SDSS_ZOFFSET,SDSS_XFOCAL,SDSS_YFOCAL,SDSS_CHUNK,SDSS_TILE,SDSS_PLATESN2,SDSS_M_I,SDSS_SN_MEDIAN_ALL,SDSS_GALEX_MATCHED,SDSS_FUV,SDSS_FUV_IVAR,SDSS_NUV,SDSS_NUV_IVAR,SDSS_UKIDSS_MATCHED,SDSS_YFLUX,SDSS_YFLUX_ERR,SDSS_JFLUX,SDSS_JFLUX_ERR,SDSS_HFLUX,SDSS_HFLUX_ERR,SDSS_KFLUX,SDSS_KFLUX_ERR,SDSS_W1_FLUX,SDSS_W1_FLUX_IVAR,SDSS_W1_MAG,SDSS_W1_MAG_ERR,SDSS_W1_CHI2,SDSS_W1_FLUX_SNR,SDSS_W1_SRC_FRAC,SDSS_W1_EXT_FLUX,SDSS_W1_EXT_FRAC,SDSS_W1_NPIX,SDSS_W2_FLUX,SDSS_W2_FLUX_IVAR,SDSS_W2_MAG,SDSS_W2_MAG_ERR,SDSS_W2_CHI2,SDSS_W2_FLUX_SNR,SDSS_W2_SRC_FRAC,SDSS_W2_EXT_FLUX,SDSS_W2_EXT_FRAC,SDSS_W2_NPIX,SDSS_FIRST_MATCHED,SDSS_FIRST_FLUX,SDSS_FIRST_SNR,SDSS_SDSS2FIRST_SEP,SDSS_JMAG,SDSS_JMAG_ERR,SDSS_JSNR,SDSS_JRDFLAG,SDSS_HMAG,SDSS_HMAG_ERR,SDSS_HSNR,SDSS_HRDFLAG,SD

## preliminary classification

GAIA

In [360]:

th = 0.9
qso_mask = desi_matched.eval('(GAIA_classprob_dsc_combmod_quasar > @th) ')
desi_matched.loc[qso_mask, 'class_GAIA_class'] = 'QSO'

galaxy_mask = desi_matched.eval('(GAIA_classprob_dsc_combmod_galaxy > @th) ')
desi_matched.loc[galaxy_mask, 'class_GAIA_class'] = 'GALAXY'



th = 3
star_mask = desi_matched.eval('(GAIA_parallax/GAIA_parallax_error > @th) | (GAIA_pmra/GAIA_pmra_error > @th) | (GAIA_pmdec/GAIA_pmdec_error > @th)')
desi_matched.loc[star_mask, 'class_GAIA_class'] = 'STAR'

#desi_matched.loc[desi_matched.class_GAIA_class.isna() & ~desi_matched.GAIA_DR3Name.isna(), 'class_GAIA_class'] = 'undef'


In [361]:
print('NOTE NANS ARE NOT PRINTER IN .value_counts()')
desi_matched.class_GAIA_class.value_counts()

NOTE NANS ARE NOT PRINTER IN .value_counts()


QSO       1482
STAR       390
GALAXY     224
Name: class_GAIA_class, dtype: int64

SDSS

In [362]:
# sdss_class_mask = desi_matched.eval('~SDSS_spCl.isna() ')
# desi_matched.loc[sdss_class_mask, 'class_SDSS_class'] = desi_matched.SDSS_spCl[sdss_class_mask]
# assert len(desi_sdss.SDSS_spCl) == np.sum(sdss_class_mask), f'SDSS_spCl and class_SDSS_class have different lengths {len(desi_sdss.SDSS_spCl)} {len(desi_matched.class_SDSS_class)}'
sdss_class_mask = desi_matched.eval('~SDSS_SDSS_NAME.isna() ')
desi_matched.loc[sdss_class_mask, 'class_SDSS_class'] = 'QSO'

In [363]:
desi_matched.class_SDSS_class.value_counts()

QSO    2017
Name: class_SDSS_class, dtype: int64

MILQ

In [364]:
mask_extragal = desi_matched.eval(" MILQ_TYPE in ['QSO', 'AGN', 'AGN-NL', 'QSO-NL', 'BLAZAR']") #GALAXY
desi_matched.loc[mask_extragal, 'class_MILQ_class'] = 'QSO'

mask_extragal = desi_matched.eval(" MILQ_TYPE in ['GALAXY']") #GALAXY
desi_matched.loc[mask_extragal, 'class_MILQ_class'] = 'GALAXY'


## Compare classes

In [365]:
desi_matched.MILQ_TYPE

0                NaN
1                QSO
2                NaN
3                NaN
4       RADIO-SOURCE
            ...     
9210             NaN
9211             NaN
9212             NaN
9213             QSO
9214             NaN
Name: MILQ_TYPE, Length: 9215, dtype: object

In [367]:
print('both SDSS and MILQ have class and they are the same:')
print(len(desi_matched.query("~class_MILQ_class.isna() & ~class_SDSS_class.isna() & class_MILQ_class==class_SDSS_class")))
print('both SDSS and MILQ have class and they are different:')
print(len(desi_matched.query("~class_MILQ_class.isna() & ~class_SDSS_class.isna() & class_MILQ_class!=class_SDSS_class")))

print('SDSS class available only but not MILQ class:')
print(len(desi_matched.query("class_MILQ_class.isna() & ~class_SDSS_class.isna()")))

print('MILQ class available only but not SDSS class:')
print(len(desi_matched.query("class_SDSS_class.isna() & ~class_MILQ_class.isna()")))

both SDSS and MILQ have class and they are the same:
1999
both SDSS and MILQ have class and they are different:
0
SDSS class available only but not MILQ class:
18
MILQ class available only but not SDSS class:
365


In [368]:
desi_matched.groupby('class_GAIA_class').class_SDSS_class.value_counts()


class_GAIA_class  class_SDSS_class
GALAXY            QSO                   10
QSO               QSO                 1214
STAR              QSO                    5
Name: class_SDSS_class, dtype: int64

In [369]:
desi_matched.groupby('class_GAIA_class').class_MILQ_class.value_counts()

class_GAIA_class  class_MILQ_class
GALAXY            QSO                   53
                  GALAXY                35
QSO               QSO                 1261
                  GALAXY                 3
STAR              QSO                   11
                  GALAXY                 1
Name: class_MILQ_class, dtype: int64

When GAIA indicate star but SDSS says QSO

In [371]:
# #[x for x in desi_gaia_sdss.columns if 'mag' in x]
# mags_cols = ['desi_dered_mag_g', 'SDSS_gmag', 'GAIA_Gmag', 'desi_dered_mag_r', 'SDSS_rmag', 'desi_dered_mag_z', 'SDSS_zmag']
# desi_matched.query('class_SDSS_class=="QSO" & class_GAIA_class=="STAR"')[mags_cols]


## Finalize classes

In [372]:
desi_matched

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,GAIA_sep,GAIA_source_id,GAIA_ra,GAIA_ra_error,GAIA_dec,GAIA_dec_error,GAIA_parallax,GAIA_parallax_error,GAIA_pmra,GAIA_pmra_error,GAIA_pmdec,GAIA_pmdec_error,GAIA_phot_g_mean_flux,GAIA_phot_g_mean_flux_error,GAIA_phot_g_mean_mag,GAIA_phot_bp_mean_flux,GAIA_phot_bp_mean_flux_error,GAIA_phot_bp_mean_mag,GAIA_phot_rp_mean_flux,GAIA_phot_rp_mean_flux_error,GAIA_phot_rp_mean_mag,GAIA_bp_rp,GAIA_classprob_dsc_combmod_quasar,GAIA_classprob_dsc_combmod_galaxy,GAIA_classprob_dsc_combmod_star,GAIA_classprob_dsc_combmod_whitedwarf,GAIA_classprob_dsc_combmod_binarystar,GAIA_spectraltype_esphs,GAIA_distance_gspphot,GAIA_distance_msc,GAIA_reliable_pmra,GAIA_reliable_pmdec,GAIA_ra_desi_epoch,GAIA_dec_desi_epoch,SDSS_sep,SDSS_CATALOG,SDSS_SDSS_NAME,SDSS_RA,SDSS_DEC,SDSS_PLATE,SDSS_MJD,SDSS_FIBERID,SDSS_AUTOCLASS_PQN,SDSS_AUTOCLASS_DR14Q,SDSS_IS_QSO_QN,SDSS_Z_QN,SDSS_RANDOM_SELECT,SDSS_Z_10K,SDSS_Z_CONF_10K,SDSS_PIPE_CORR_10K,SDSS_IS_QSO_10K,SDSS_THING_ID,SDSS_Z_VI,SDSS_Z_CONF,SDSS_CLASS_PERSON,SDSS_Z_DR12Q,SDSS_IS_QSO_DR12Q,SDSS_Z_DR7Q_SCH,SDSS_IS_QSO_DR7Q,SDSS_Z_DR6Q_HW,SDSS_Z_DR7Q_HW,SDSS_IS_QSO_FINAL,SDSS_Z,SDSS_SOURCE_Z,SDSS_Z_PIPE,SDSS_ZWARNING,SDSS_OBJID,SDSS_Z_PCA,SDSS_ZWARN_PCA,SDSS_DELTACHI2_PCA,SDSS_Z_HALPHA,SDSS_ZWARN_HALPHA,SDSS_DELTACHI2_HALPHA,SDSS_Z_HBETA,SDSS_ZWARN_HBETA,SDSS_DELTACHI2_HBETA,SDSS_Z_MGII,SDSS_ZWARN_MGII,SDSS_DELTACHI2_MGII,SDSS_Z_CIII,SDSS_ZWARN_CIII,SDSS_DELTACHI2_CIII,SDSS_Z_CIV,SDSS_ZWARN_CIV,SDSS_DELTACHI2_CIV,SDSS_Z_LYA,SDSS_ZWARN_LYA,SDSS_DELTACHI2_LYA,SDSS_Z_LYAWG,SDSS_BAL_PROB,SDSS_BI_CIV,SDSS_ERR_BI_CIV,SDSS_AI_CIV,SDSS_ERR_AI_CIV,SDSS_BI_SIIV,SDSS_ERR_BI_SIIV,SDSS_AI_SIIV,SDSS_ERR_AI_SIIV,SDSS_BOSS_TARGET1,SDSS_EBOSS_TARGET0,SDSS_EBOSS_TARGET1,SDSS_EBOSS_TARGET2,SDSS_ANCILLARY_TARGET1,SDSS_ANCILLARY_TARGET2,SDSS_NSPEC_SDSS,SDSS_NSPEC_BOSS,SDSS_NSPEC,SDSS_SKYVERSION,SDSS_RUN_NUMBER,SDSS_RERUN_NUMBER,SDSS_CAMCOL_NUMBER,SDSS_FIELD_NUMBER,SDSS_ID_NUMBER,SDSS_LAMBDA_EFF,SDSS_ZOFFSET,SDSS_XFOCAL,SDSS_YFOCAL,SDSS_CHUNK,SDSS_TILE,SDSS_PLATESN2,SDSS_M_I,SDSS_SN_MEDIAN_ALL,SDSS_GALEX_MATCHED,SDSS_FUV,SDSS_FUV_IVAR,SDSS_NUV,SDSS_NUV_IVAR,SDSS_UKIDSS_MATCHED,SDSS_YFLUX,SDSS_YFLUX_ERR,SDSS_JFLUX,SDSS_JFLUX_ERR,SDSS_HFLUX,SDSS_HFLUX_ERR,SDSS_KFLUX,SDSS_KFLUX_ERR,SDSS_W1_FLUX,SDSS_W1_FLUX_IVAR,SDSS_W1_MAG,SDSS_W1_MAG_ERR,SDSS_W1_CHI2,SDSS_W1_FLUX_SNR,SDSS_W1_SRC_FRAC,SDSS_W1_EXT_FLUX,SDSS_W1_EXT_FRAC,SDSS_W1_NPIX,SDSS_W2_FLUX,SDSS_W2_FLUX_IVAR,SDSS_W2_MAG,SDSS_W2_MAG_ERR,SDSS_W2_CHI2,SDSS_W2_FLUX_SNR,SDSS_W2_SRC_FRAC,SDSS_W2_EXT_FLUX,SDSS_W2_EXT_FRAC,SDSS_W2_NPIX,SDSS_FIRST_MATCHED,SDSS_FIRST_FLUX,SDSS_FIRST_SNR,SDSS_SDSS2FIRST_SEP,SDSS_JMAG,SDSS_JMAG_ERR,SDSS_JSNR,SDSS_JRDFLAG,SDSS_HMAG,SDSS_HMAG_ERR,SDSS_HSNR,SDSS_HRDFLAG,SD

In [374]:

def assign_final_class(row):
    if not pd.isna(row.class_MILQ_class):
        return row.class_MILQ_class
    elif not pd.isna(row.class_SDSS_class):
        return row.class_SDSS_class
    elif not pd.isna(row.class_GAIA_class):
        return row.class_GAIA_class
    else:
        return 'unknown'

def assign_final_redshift(row):
    if not pd.isna(row.MILQ_Z):
        return row.MILQ_Z
    elif not pd.isna(row.SDSS_Z):
        return row.SDSS_Z
    else:
        return np.nan

desi_matched.loc[:, 'class_final'] = desi_matched.apply(assign_final_class, axis=1)
desi_matched.loc[:, 'redshift_final'] = desi_matched.apply(assign_final_redshift, axis=1)


num_qso = len(desi_matched.query('class_MILQ_class=="QSO"'))
num_qso += len(desi_matched.query('class_SDSS_class=="QSO" & class_MILQ_class.isna()'))
num_qso += len(desi_matched.query('class_SDSS_class.isna() & class_MILQ_class.isna() & class_GAIA_class=="QSO"'))
desi_matched.class_final.value_counts()['QSO'] == num_qso, f'QSO count is wrong, should be {num_qso}, got {desi_matched.class_final.value_counts()["QSO"]}'
desi_matched.class_final.value_counts()

unknown    6105
QSO        2531
STAR        378
GALAXY      201
Name: class_final, dtype: int64

In [375]:
desi_matched.to_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match_xmatch.pkl')